In [ ]:
# преобразование напрямую в датафрейм библиотеки Pandas

# импортируем библиотеку
import pandas as pd

# применим функцию read_csv() и посмотрим на первые три записи файла train.csv
train = pd.read_csv('/content/train.csv')
train.head(3)
# сделаем то же самое с файлом test.csv
test = pd.read_csv('/content/test.csv')
test.head(3)

train.info()

# для построения графиков воспользуемся новой для нас библиотекой seaborn
import seaborn as sns

# посмотрим насколько значим класс билета для выживания пассажира
# с помощью x и hue мы можем уместить две категориальные переменные на одном графике
sns.countplot(x = 'Pclass', hue = 'Survived', data = train)

In [ ]:
# кто выживал чаще, мужчины или женщины?
sns.countplot(x = 'Sex', hue = 'Survived', data = train)

In [ ]:
# выявим пропущенные значения с помощью .isnull() и посчитаем их количество sum()
train.isnull().sum()

In [ ]:
# переменная Cabin (номер каюты), скорее всего, не является самой важной
# избавимся от нее с помощью метода .drop()
# (параметр axis отвечает за столбцы, inplace = True сохраняет изменения)
train.drop(columns = 'Cabin', axis = 1, inplace = True)
# а вот Age (возраст) точно важен, заменим пустые значения средним арифметическим
train['Age'].fillna(train['Age'].mean(), inplace = True)
# у нас остаются две пустые строки в Embarked, удалим их
train.dropna(inplace = True)
train.isnull().sum()

In [ ]:
# применим one-hot encoding к переменной Sex (пол) с помощью метода .get_dummies()
pd.get_dummies(train['Sex']).head(3)
# удалим первый столбец, он избыточен
sex = pd.get_dummies(train['Sex'], drop_first = True)
sex.head(3)

# Сделаем то же самое для переменных Pclass и Embarked
embarked = pd.get_dummies(train['Embarked'], drop_first = True)
pclass = pd.get_dummies(train['Pclass'], drop_first = True)

# Присоединим новые (закодированные) переменные к исходному датафрейму train.
train = pd.concat([train, pclass, sex, embarked], axis = 1)

In [ ]:
# Отбор признаков

# В первую очередь, удалим исходные (до применения one-hot encoding)
# переменные Sex, Pclass и Embarked
# Кроме того, переменные PassengerId, Name и Ticket вряд ли скажут что-то
# определенное о шансах на выживание пассажира, удалим и их

# применим функцию drop() к соответствующим столбцам
train.drop(['PassengerId', 'Pclass', 'Name', 'Sex', 'Ticket', 'Embarked'], axis = 1, inplace = True)
train.head(3)

In [ ]:
# Нормализация данных

# импортируем класс StandardScaler
from sklearn.preprocessing import StandardScaler

# создадим объект этого класса
scaler = StandardScaler()

# выберем те столбцы, которые мы хотим масштабировать
cols_to_scale = ['Age', 'Fare']

# рассчитаем среднее арифметическое и СКО для масштабирования данных
scaler.fit(train[cols_to_scale])

# применим их
train[cols_to_scale] = scaler.transform(train[cols_to_scale])

# посмотрим на результат
train.head(3)

In [ ]:
# Переменные 2 и 3 (второй и третий класс) выражены числами, а не строками

train.columns
train.columns = train.columns.map(str)

In [ ]:
# Разделение обучающей выборки на признаки и целевую переменную
# поместим в X_train все кроме столбца Survived
X_train = train.drop('Survived', axis = 1)

# столбец 'Survived' станет нашей целевой переменной (y_train)
y_train = train['Survived']

X_train.head(3)

In [ ]:
# Обучение модели логистической регрессии

# импортируем логистическую регрессию из модуля linear_model библиотеки sklearn
from sklearn.linear_model import LogisticRegression

# создадим объект этого класса и запишем его в переменную model
model = LogisticRegression()

# обучим нашу модель
model.fit(X_train, y_train)

In [ ]:
# сделаем предсказание класса на обучающей выборке
y_pred_train = model.predict(X_train)

# построим матрицу ошибок
from sklearn.metrics import confusion_matrix

# передадим ей фактические и прогнозные значения
conf_matrix = confusion_matrix(y_train, y_pred_train)

# преобразуем в датафрейм
conf_matrix_df = pd.DataFrame(conf_matrix)
conf_matrix_df

# Для удобства интерпретации добавим подписи.
conf_matrix_labels = pd.DataFrame(conf_matrix, columns = ['Прогноз погиб', 'Прогноз выжил'], index = ['Факт погиб', 'Факт выжил'])
conf_matrix_labels


In [ ]:
"""посмотрим на метрику accuracy. Она показывает долю правильно предсказанных значений. То есть мы берем тех, кого верно предсказали как погибших (true negative, TN, таких было 478), и тех, кого верно предсказали как выживших (true positive, TP, 237), и делим на общее число прогнозов."""

# рассчитаем метрику accuracy вручную
round((478 + 237)/(478 + 237 + 71 + 103), 3)

In [ ]:
# импортируем метрику accuracy из sklearn
from sklearn.metrics import accuracy_score

# так же передадим ей фактические и прогнозные значения
model_accuracy = accuracy_score(y_train, y_pred_train)

# округлим до трех знаков после запятой
round(model_accuracy, 3)

In [ ]:
# Построение прогноза на тестовых данных

test.info()

"""Взглянув на сводку по тестовым данным, становится заметна одна сложность. Мы обучили модель на обработанных данных. В частности, мы заполнили пропуски, закодировали категориальные переменные и убрали лишние признаки. Кроме того, мы масштабировали количественные переменные и превратили названия столбцов в строки.

Для того чтобы наша модель смогла работать с тестовой выборкой нам нужно таким же образом обработать и эти данные.
При надо обратить внимание, мы не нарушаем принципа разделения данных, поскольку меняем тестовую выборку так же, как мы меняли обучающую."""

# для начала дадим датасету привычное название X_test
X_test = test
# заполним пропуски в переменных Age и Fare средним арифметическим
X_test['Age'].fillna(test['Age'].mean(), inplace = True)
X_test['Fare'].fillna(test['Fare'].mean(), inplace = True)

# выполним one-hot encoding категориальных переменных
sex = pd.get_dummies(X_test['Sex'], drop_first = True)
embarked = pd.get_dummies(X_test['Embarked'], drop_first = True)
pclass = pd.get_dummies(X_test['Pclass'], drop_first = True)

# присоединим новые столбцы к исходному датафрейму
X_test = pd.concat([test, pclass, sex, embarked], axis = 1)

# и удалим данные, которые теперь не нужны
X_test.drop(['PassengerId', 'Pclass', 'Name', 'Sex', 'Cabin', 'Ticket', 'Embarked'], axis = 1, inplace = True)

# посмотрим на результат
X_test.head(3)

In [ ]:
"""Теперь нужно масштабировать количественные переменные. Для этого мы будем использовать те параметры (среднее арифметическое и СКО), которые мы получили при обработке обучающей выборки. Так мы сохраним единообразие изменений и избежим утечки данных (data leakage)."""

# применим среднее арифметическое и СКО обучающей выборки для масштабирования тестовых данных
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])
X_test.head(3)

# Превращаем название столбцов в строки
X_test.columns = X_test.columns.map(str)

# Делаем прогноз на тестовой выборке
y_pred_test = model.predict(X_test)

# посмотрим на первые 10 прогнозных значений
y_pred_test[:10]

In [ ]:
# Сохранение результата в новом файле на сервере

# возьмем индекс пассажиров из столбца PassengerId тестовой выборки
ids = test['PassengerId']
 
# создадим датафрейм из словаря, в котором
# первая пара ключа и значения - это id пассажира, вторая - прогноз "на тесте"
result = pd.DataFrame({'PassengerId': ids, 'Survived': y_pred_test})
 
# посмотрим, что получилось
result.head()
# создадим новый файл result.csv с помощью функции to_csv(), удалив при этом индекс
result.to_csv('result.csv', index = False)
 
# файл будет сохранен в 'Сессионном хранилище' и, если все пройдет успешно, выведем следующий текст:
print('Файл успешно сохранился в сессионное хранилище!')

In [ ]:
# скачать файл на жесткий диск.
# применим метод .download() объекта files
from google.colab import files
files.download('/content/result.csv')